# Evaluation using BERTscore

[Source](https://huggingface.co/spaces/evaluate-metric/bertscore)

In [1]:
import pandas as pd

df = pd.read_csv("../../data/processed/20231220_metrics_CAUSAL.csv")
df.head()

,input,target,sesgo_pronombre,sesgo_otro,seq2seq_document,causal_document,generation,reference_tokens,max_ref_len,generated_tokens,input_tokens,bleu_gen,bleu_input,bleu_dif,rouge
0,Estimada comunidad beauchefiana: ¿Tienes papel...,['Estimada comunidad beauchefiana: ¿Tienes pap...,NO,NO,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Estimada', 'comunidad', 'beauchefiana', ':'...",11,"['Estimada', 'comunidad', 'beauchefiana', ':',...","['Estimada', 'comunidad', 'beauchefiana', ':',...",1.0,1.0,0.0,1.0
1,Desde hoy y hasta el 19 de diciembre puedes de...,['Desde hoy y hasta el 19 de diciembre puedes ...,NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'd...",17,"['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'de...","['Desde', 'hoy', 'y', 'hasta', 'el', '19', 'de...",1.0,1.0,0.0,1.0
2,Revisa en el afiche qué tipo de papeles puedes...,['Revisa en el afiche qué tipo de papeles pued...,NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Revisa', 'en', 'el', 'afiche', 'qué', 'tipo...",24,"['Revisa', 'en', 'el', 'afiche', 'qué', 'tipo'...","['Revisa', 'en', 'el', 'afiche', 'qué', 'tipo'...",1.0,1.0,0.0,1.0
3,Estimada Comunidad: La Subdirección de Puebl...,['Estimada Comunidad: La Subdirección de Pue...,NO,NO,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Estimada', 'Comunidad', ':', 'La', 'Subdire...",35,"['Estimada', 'Comunidad', ':', 'La', 'Subdirec...","['Estimada', 'Comunidad', ':', 'La', 'Subdirec...",1.0,1.0,0.0,1.0
4,"Postulaciones, labores y más información en: ...","['Postulaciones, labores y más información en:...",NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,<human>: ¿Puedes reescribir el siguiente texto...,"[['Postulaciones', ',', 'labores', 'y', 'más',...",18,"['Postulaciones', ',', 'labores', 'y', 'más', ...","['Postulaciones', ',', 'labores', 'y', 'más', ...",1.0,1.0,0.0,1.0


In [2]:
import ast
from collections import Counter

df['target'] = df['target'].apply(lambda text: ast.literal_eval(text))
Counter([len(t) for t in df['target']])

Counter({1: 720, 7: 16, 6: 15, 8: 13, 5: 6, 2: 5, 4: 4, 3: 3})

Notice that the target either keeps the original text or corrects the bias in one of the several possible ways. We will separate the dataframe in a way that each one of them becomes a separate candidate for the evaluation.

In [3]:
df['ID_row'] = df.index  # column to keep track of the original row
df = df.explode('target', ignore_index=True)

In [4]:
get_gen = lambda text: text.split('\n')[2].replace('    <assistant>: ','')
df['output'] = df['generation'].apply(get_gen)

### Example

There are potentially many options for replacing a biased text. We will consider the highest similarity among all candidate options.

In [5]:
references = ['Estimad@s estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimados/as estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimadas/os estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimados y estimadas estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimadas y estimados estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimados(as) estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.', 'Estimadas(os) estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.']
predictions = ['Estimada Comunidad de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.' for _ in references]

In [6]:
from evaluate import load
bertscore = load("bertscore")

In [7]:
results = bertscore.compute(predictions=predictions, references=references, model_type="bert-base-multilingual-cased")
print(results)

{'precision': [0.9773577451705933, 0.9740340709686279, 0.9773645997047424, 0.9707762598991394, 0.9730334877967834, 0.9715868234634399, 0.9754259586334229], 'recall': [0.9639482498168945, 0.9562501907348633, 0.9597264528274536, 0.9487558603286743, 0.952041506767273, 0.9467052221298218, 0.9509559869766235], 'f1': [0.9706066846847534, 0.9650602340698242, 0.9684652090072632, 0.9596397876739502, 0.9624230265617371, 0.9589846134185791, 0.963035523891449], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.47.1)'}


The results are very similar since the difference is only a few tokens. We'll subtract the value obtained between the original input (potentially biased) and the candidates. Notice that when the input is unbised then the strings should coincide (making the BERTscore, ROUGE, BLEU values equal to zero).

In [8]:
inputs = ['Estimados estudiantes de Pregrado,Junto con saludar les invitamos al OPEN MDS, charla Abierta para conocer los detalles del Magíster en Ciencia de Datos MDS de nuestra Facultad el cual es articulable con las carreras de pregrado FCFM.' for _ in references]
results_input = bertscore.compute(predictions=inputs, references=references, model_type="bert-base-multilingual-cased")

In [9]:
[results['f1'][i]-results_input['f1'][i] for i in range(len(results['f1']))]

[-0.01429903507232666,
 -0.020494282245635986,
 -0.01409846544265747,
 -0.023262202739715576,
 -0.016454339027404785,
 -0.025037288665771484,
 -0.017160356044769287]

Full evaluation

In [10]:
results = bertscore.compute(predictions=df['output'], references=df['target'], model_type="bert-base-multilingual-cased")

In [11]:
df['precision'] = results['precision']
df['recall'] = results['recall']
df['f1'] = results['f1']

In [12]:
def filter_best_scores(df, id_column, score_column):
    """
    Reduces the dataframe to only the rows with the best score for each ID.

    Parameters:
    - df (pd.DataFrame): The original dataframe
    - id_column (str): Name of the column containing unique IDs
    - score_column (str): Name of the column containing the scores

    Returns:
    - pd.DataFrame: A filtered dataframe with the best score per ID
    """
    # Find the maximum score for each ID_row
    best_scores = df.groupby(id_column)[score_column].transform('max')
    
    # Filter the rows where the score matches the maximum score for each ID_row
    filtered_df = df[df[score_column] == best_scores].copy(deep=True)
    
    return filtered_df

In [13]:
df_results = filter_best_scores(df, 'ID_row', 'f1')

In [14]:
import numpy as np

for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f'\n\tmean: {np.mean(df_results[k])}\n\tstd: {np.std(df_results[k])}\n')

precision 
	mean: 0.9984012772817441
	std: 0.008954700796637778

recall 
	mean: 0.997340931459461
	std: 0.0178454148136428

f1 
	mean: 0.9978308397943102
	std: 0.013742493489760968



Subtracting input

In [15]:
results_input = bertscore.compute(
    predictions=df_results['input'],
    references=df_results['target'], model_type="bert-base-multilingual-cased")
df_results['precision_input'] = results_input['precision']
df_results['recall_input'] = results_input['recall']
df_results['f1_input'] = results_input['f1']

df_results['precision_diff'] = df_results['precision'] - df_results['precision_input']
df_results['recall_diff'] = df_results['recall'] - df_results['recall_input']
df_results['f1_diff'] = df_results['f1'] - df_results['f1_input']

In [16]:
for k in results.keys():
    if not k == 'hashcode':
        print(f'{k}',f"\n\tmean (diff): {np.mean(df_results[k+'_diff'])}\n\tstd (diff): {np.std(df_results[k+'_diff'])}\n")

precision 
	mean (diff): -0.000559201325906817
	std (diff): 0.00801560095078384

recall 
	mean (diff): -0.00044103915734059363
	std (diff): 0.0179361198875062

f1 
	mean (diff): -0.0005302721886988491
	std (diff): 0.01339388807917294

